In [4]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [5]:
data_path = "/Users/yu.zadorozhnaya/Machine-Learning-and-Predictive-Analytics-MSU-/machine_learning_04_data"
train = pd.read_csv(os.path.join(data_path,"train_sample.csv"))
test = pd.read_csv(os.path.join(data_path,"test_sample.csv"))

In [6]:
test.head()

,X1,X2,X3,X4,X5,X6,X7
0,0.871771,0.905350,0.555100,0.239740,0.976275,0.107325,0.826784
1,0.092639,0.934172,0.127539,0.790953,0.052702,0.389997,0.766328
2,0.769920,0.932810,0.970285,0.000200,0.416587,0.911954,0.209831
3,0.249039,0.059093,0.479781,0.614687,0.030063,0.468889,0.310273
4,0.398352,0.108593,0.088371,0.819770,0.449875,0.989026,0.249356


In [7]:
train.head()

,X1,X2,X3,X4,X5,X6,X7,class
0,0.652472,0.204031,0.587416,0.147057,0.646242,0.927309,0.500918,1
1,0.151208,0.075754,0.952729,0.917972,0.714318,0.882891,0.325293,0
2,0.654684,0.610544,0.039420,0.049184,0.635003,0.766042,0.972134,1
3,0.446010,0.387978,0.310912,0.911122,0.744979,0.416282,0.402164,0
4,0.781609,0.112714,0.088151,0.622971,0.127196,0.245402,0.619664,0


In [8]:
y = train["class"]
X = train.drop("class", axis=1)

In [9]:
param = {'objective': 'binary:logistic', 'eval_metric': 'auc'}

In [10]:
dtrain = xgb.DMatrix(X, label=y)
m = xgb.cv(param, dtrain, nfold = 5,num_boost_round=50)
m.head()

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.969278,0.002957,0.913147,0.021896
1,0.986247,0.001445,0.918729,0.026632
2,0.990329,0.001958,0.922981,0.026175
3,0.992868,0.000971,0.927300,0.022272
4,0.995010,0.000748,0.925574,0.024126


In [11]:
bestNR = np.argmax(m['test-auc-mean'])
bestAUC = m.loc[bestNR,'test-auc-mean']
print("Best number of rounds = %i; AUC = %f" %(bestNR, bestAUC))

Best number of rounds = 49; AUC = 0.945656


In [12]:
best_xgb = xgb.train(param, dtrain, num_boost_round=bestNR)

In [13]:
prediction = best_xgb.predict(xgb.DMatrix(test))
prediction.shape

(500,)

In [14]:
RF = RandomForestClassifier(random_state=0).fit(X, y)
t = pd.Series(RF.feature_importances_, index=range(1,8))

In [15]:
rf_most_important = t.sort_values(ascending=False).index
rf_most_important

Index([2, 4, 5, 3, 1, 7, 6], dtype='int64')

In [16]:
np.savez('submission.npz', rf_most_important=rf_most_important[0], 
         prediction=prediction)

In [17]:
npzfile = np.load('submission.npz')
print(npzfile['rf_most_important'],npzfile['prediction'].shape)

2 (500,)
